# S&P 2019 Data Exploration Stage

**by Marc Angelo Acebedo**

# Table of Contents
- [Introduction](#introduction)
- [Data Setup & Overview](#setup)
- [Univariate](#uni)
- [Bivariate](#bi)
- [Multivariate](#multi)


# I) Introduction <a id = "introduction"> </a>


- I kept features in separate CSVs because date formats differ. `eps_fc` and `eps_act` 

NameError: name 'yearly_avgs' is not defined

## II) Data Setup & Overview<a id = "setup"></a> 

In [2]:
#import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline
plt.style.use('bmh')

In [3]:
#define data directories
PATH_CLEAN = './data/clean/'
PATH_CLEAN_AVGS = './data/clean/averages/'

In [4]:
#define visuals destination
PATH_VISUALS = './visuals/'

**Import features and all averages**

In [11]:
features = pd.read_csv(PATH_CLEAN + 'features.csv', low_memory = False)
avgs = pd.read_csv(PATH_CLEAN_AVGS + 'avgs.csv')

## III) Univariate Exploration <a id="uni"> </a>

AttributeError: module 'matplotlib.pyplot' has no attribute 'barchart'

## IV) Bivariate Exploration<a id = "bi"></a>

## V) Multivariate Exploration <a id="multi"></a>